In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
import numpy as np
import import_ipynb
import basefile as bf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
class BaggingEnsembleClassifier: #gpt
    def __init__(self, base_classifier, num_estimators, fraction_samples, bootstrap, voting):
        self.base_classifier = base_classifier
        self.num_estimators = num_estimators
        self.estimators = []
        self.sample_indices = []
        self.fraction_samples = fraction_samples
        self.bootstrap = bootstrap
        self.voting = voting
    def fit(self, X, y):        
        X_reset = X.reset_index(drop=True) if isinstance(X, pd.DataFrame) else X
        y_reset = y.reset_index(drop=True) if isinstance(y, pd.Series) else y
        for nums in range(self.num_estimators):
            if self.bootstrap:
                indices = resample(np.arange(len(X_reset)), n_samples=int(self.fraction_samples * len(X_reset)))
            else:
                indices = np.random.choice(np.arange(len(X_reset)), size=int(self.fraction_samples * len(X_reset)), replace=False)
            self.sample_indices.append(indices)
            X_sampled, y_sampled = X_reset[indices], y_reset[indices]
            base_model = None
            if self.base_classifier == "DT":
                base_model = DecisionTreeClassifier(max_depth=25, criterion="gini") 
            elif self.base_classifier == "LR":
                base_model = bf.MultinomialLogisticRegression(learning_rate = 0.1, max_iter = 2000)
            elif self.base_classifier == "MLP":
                base_model = bf.MLPClassifier(input_size=X.shape[1], hidden_layers=[10, 15], learning_rate=0.01, activation='tanh', epoch = 2000)
            base_model.fit(X_sampled, y_sampled)
            self.estimators.append(base_model)
    def predict(self, X):       
        if self.voting == 'HARD':
            if self.base_classifier == "MLP":
                estm = []
                for estimators in self.estimators:
                    estm.append(estimators.predict(X).flatten())
                y_pred = []
                for i in range(len(X)):
                    y_pred.append(np.argmax(np.bincount([est[i] for est in estm])))
                return np.array(y_pred)
            predictions = [estimator.predict(X) for estimator in self.estimators]
            predictions = np.array(predictions).T
            y_pred = np.array([np.argmax(np.bincount(class_prediction)) for class_prediction in predictions])
            return y_pred
        elif self.voting == 'SOFT' and self.base_classifier == "MLP":
            predictions = []
            confidences = []
            for estimator in self.estimators:
                predictions.append(estimator.predict(X).flatten())
                confidences.append(np.max(estimator.predict_proba(X), axis=1))
            predictions = np.array(predictions).T
            confidences = np.array(confidences).T
            y_pred = []
            for prediction, confidence in zip(predictions, confidences):
                class_weights = {}
                for pred, conf in zip(prediction, confidence):
                    pred = int(pred)  # Convert to integer or another hashable type
                    if pred not in class_weights:
                        class_weights[pred] = 0
                    class_weights[pred] += conf
                y_pred.append(max(class_weights, key=class_weights.get))
            return np.array(y_pred)
        elif self.voting == 'SOFT':
            predictions = [estimator.predict(X) for estimator in self.estimators]
            confidences = [np.max(estimator.predict_proba(X), axis=1) for estimator in self.estimators]
            predictions = np.array(predictions).T
            confidences = np.array(confidences).T
            y_pred = []
            for prediction, confidence in zip(predictions, confidences):
                class_weights = {}
                for pred, conf in zip(prediction, confidence):
                    if pred not in class_weights:
                        class_weights[pred] = 0
                    class_weights[pred] += conf
                y_pred.append(max(class_weights, key=class_weights.get))
            return np.array(y_pred)

In [ ]:
class BaggingEnsembleRegression:
    def __init__(self, base_model , num_estimators, fraction_samples, bootstrap, voting='hard'):
        self.base_model = base_model
        self.num_estimators = num_estimators
        self.fraction_samples = fraction_samples
        self.bootstrap = bootstrap
        self.voting = voting
        self.estimators = []
        self.sample_indices = []
    def fit(self, X, y):# gpt
        X_reset = X.reset_index(drop=True) if isinstance(X, pd.DataFrame) else X
        y_reset = y.reset_index(drop=True) if isinstance(y, pd.Series) else y
        for nums in range(self.num_estimators):
            if self.bootstrap:
                indices = resample(np.arange(len(X_reset)), n_samples=int(self.fraction_samples * len(X_reset)))
            else:
                indices = np.random.choice(np.arange(len(X_reset)), size=int(self.fraction_samples * len(X_reset)), replace=False)
            self.sample_indices.append(indices)
            X_sampled, y_sampled = X_reset[indices], y_reset[indices]
            base_model = None
            if self.base_model == "DT":
                base_model = bf.DecisionTreeRegression(max_depth=50, criterion="absolute_error")
            elif self.base_model == "LR":
                base_model = bf.LinearRegression(learning_rate = 0.1, n_iterations = 2000)
            elif self.base_model == "MLP":
                base_model = bf.MLPRegressor(input_size=X.shape[1], hidden_layers=[10, 15], learning_rate=0.01, activation='tanh', epoch = 2000)             
            base_model.fit(X_sampled, y_sampled)
            # Estimate Confidence on the validation set!
            base_model.confidence = np.mean(np.abs(base_model.predict(X_sampled).reshape(-1,) - y_sampled))
            self.estimators.append(base_model)
    def predict(self, X):
        predictions = [estimator.predict(X) for estimator in self.estimators]
        predictions = np.array(predictions)
        if self.voting == 'SOFT':
            weights = [math.exp(-estimator.confidence) for estimator in self.estimators]
            return np.average(predictions, axis=0, weights=weights)
        elif self.voting == 'HARD':
            return np.mean(predictions, axis=0)